# Semantic Search on a Corpus

In [1]:
# Import Packages
import json
import numpy as np
import pandas as pd
import tiktoken
from joblib import load

## Import Data

In [2]:
# Import Data
filepath = "C:\\Users\\t_zim\\Desktop\\Data\\Bible\\"
filename = "KJV_chapter_search.json"

df = pd.read_json(str(filepath)+str(filename))
len(df)

1198

In [3]:
# ================== #
# Tokenize Text
# ================== #

# TODO: change tiktoken out to real tokenizer... or use tiktoken

def get_tokens(text_2_encode: str, tokenizer=None):
    """
    Tokenize text in a string.

    Initialize a tokenizer if tokenizer == None.
    """

    if tokenizer is None:
        tokenizer = tiktoken.encoding_for_model("text-davinci-003")
    return tokenizer.encode(text=text_2_encode)


def get_num_tokens(text_2_encode: str, **kwargs):
    """
    Count the number of tokens in a string.
    """
    return len(get_tokens(text_2_encode=text_2_encode, **kwargs))


# ================== #
#  Get Embeddings
# ================== #

def get_embeddings(text=None, model=None):
    """
    Generate embeddings on a string of text.
    """
    if model==None:
        model = load('./model/SentBERTmodel.pkl')

    return model.encode(text)


# ================== #
#  Calculate Vector Similarity
# ================== #

def vector_similarity(x: "list[float]", y: "list[float]") -> float:
    """
    Returns the similarity between two vectors.

    Because embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


# ================== #
#  Order Chunks by Similarity
# ================== #

def measure_embedding_similarity(
    query: str,
    embeddings
):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embeddings(query)

    return [vector_similarity(query_embedding, embedding) for embedding in embeddings]


# ================== #
#  Get Similar Texts
# ================== #

def get_similar_texts(df, k):
    """
    Slice a dataframe on the top k results.  Sort the sliced dataframe descending on similarity score.

    If there are repeated results in top 5, keep them all.
    """
    response = df.nlargest(k, columns=['similarity score'],keep='all')
    response = response.sort_values(by='similarity score', ascending=False)
    return response

In [4]:
# ================== #
#  Run (score.py)
# ================== #

def run(question: str, k: int, embeddings, df) -> dict[str,str]:

    # Retrieve Top K Most Similar Results
    df['similarity score'] = measure_embedding_similarity(question, embeddings)

    # Count number of tokens in each article
    df['token count'] = df['text'].apply(get_num_tokens)
    
    # Return Chunks With Highest Similarity (Text)
    response = get_similar_texts(df, k)

    # Remove embeddings column
    keep_columns = ['book', "chapter", 'text', 'token count', 'similarity score']
    response = response[keep_columns]
    
    return response

## Ask Questions

In [5]:
response = run("who is my father?", 3, df['embeddings'], df)

response.style.set_properties(subset=['text'], **{'width': '900px'})

## Output Data (if needed)

In [6]:
"""

del df['similarity score']

OUTPUT_filepath = "C:\\Users\\hlmq\\OneDrive - Chevron\\Desktop\\Projects\\Company Relationships\\Data\\Exxon\\"
OUTPUT_file = "CompanyProfileNewsData_withEmbeddings.json"

df.to_json(str(OUTPUT_filepath)+str(OUTPUT_file), orient='records')

"""

'\n\ndel df[\'similarity score\']\n\nOUTPUT_filepath = "C:\\Users\\hlmq\\OneDrive - Chevron\\Desktop\\Projects\\Company Relationships\\Data\\Exxon\\"\nOUTPUT_file = "CompanyProfileNewsData_withEmbeddings.json"\n\ndf.to_json(str(OUTPUT_filepath)+str(OUTPUT_file), orient=\'records\')\n\n'